In [2]:
import tensorflow as tf
from pathlib import Path
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# --- Configuration ---
ROOT = Path(r"C:\Users\ARYAN\swastha\CP-AnemiC")
IMG_SIZE = 224
BATCH_SIZE = 32
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

# --- Load Datasets ---
train_ds = tf.keras.utils.image_dataset_from_directory(
    ROOT,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='int'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    ROOT,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='int'
)

# --- Calculate Class Weights ---
train_labels = np.concatenate([y for x, y in train_ds], axis=0)
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weight_dict = dict(enumerate(class_weights))
print(f"Class Weights Calculated: {class_weight_dict}")

# --- Data Augmentation Layer ---
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
])

# --- Configure Datasets for Performance ---
# We no longer apply our own rescaling or white balancing.
# The model will do it internally.
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

print("\nSimplified and stabilized data pipeline is ready.")



Found 710 files belonging to 2 classes.
Using 568 files for training.
Found 710 files belonging to 2 classes.
Using 142 files for validation.
Class Weights Calculated: {0: 0.8328445747800587, 1: 1.251101321585903}

Simplified and stabilized data pipeline is ready.


In [3]:
import tensorflow as tf

# --- Model Definition with Built-in Preprocessing ---
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Define the input layer
inputs = tf.keras.Input(shape=IMG_SHAPE)

# Apply data augmentation
x = data_augmentation(inputs)

# Use the official EfficientNetV2 preprocessing layer
# This is much more stable than our custom function.
x = tf.keras.applications.efficientnet_v2.preprocess_input(x)

# Load the base model, ensuring it's not trainable initially
base_model = tf.keras.applications.EfficientNetV2B0(
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

# Run the preprocessed data through the base model
x = base_model(x, training=False)

# Add our custom classification head
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)

# --- Compile the Model ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# --- Define a more patient EarlyStopping callback ---
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc',
    patience=8, # More patience before stopping
    mode='max',
    restore_best_weights=True
)

# --- Train the Model ---
print("--- Starting Stable Training ---")
history = model.fit(
    train_ds,
    epochs=50,
    validation_data=val_ds,
    class_weight=class_weight_dict,
    callbacks=[early_stopping]
)

print("\n--- Training Complete ---")

# --- Final Evaluation ---
loss, accuracy, auc = model.evaluate(val_ds)
print(f"\nFinal Validation Accuracy: {accuracy*100:.2f}%")
print(f"Final Validation AUC: {auc:.4f}")



--- Starting Stable Training ---
Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 18s 552ms/step - accuracy: 0.5211 - auc: 0.4770 - loss: 0.7389 - val_accuracy: 0.4648 - val_auc: 0.4930 - val_loss: 0.7279
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 291ms/step - accuracy: 0.4683 - auc: 0.4884 - loss: 0.7286 - val_accuracy: 0.5211 - val_auc: 0.5238 - val_loss: 0.6980
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 295ms/step - accuracy: 0.5211 - auc: 0.5329 - loss: 0.7082 - val_accuracy: 0.5282 - val_auc: 0.5703 - val_loss: 0.6824
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 341ms/step - accuracy: 0.5264 - auc: 0.5265 - loss: 0.7122 - val_accuracy: 0.5563 - val_auc: 0.6066 - val_loss: 0.6753
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 364ms/step - accuracy: 0.5704 - auc: 0.5896 - loss: 0.6850 - val_accuracy: 0.5704 - val_auc: 0.6441 - val_loss: 0.6662
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 294ms/step - accuracy: 0.5757 - auc: 0.5912 - loss: 0.6846 - val_accuracy: 0.5986 - val_auc: 0.6512 - val_loss: 0.6699
Epoch 

KeyboardInterrupt: 

In [4]:
import tensorflow as tf
import numpy as np

# --- 1. Learning Rate Schedule with Warm-up ---
# This is a more advanced learning rate schedule that can improve stability.
initial_learning_rate = 0.001
# The number of steps for the warm-up phase
warmup_steps = int(0.1 * (len(train_ds) * 50)) # 10% of total steps for 50 epochs

# Create a learning rate schedule with a cosine decay and warm-up
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate,
    decay_steps=(len(train_ds) * 50) - warmup_steps, # Total steps minus warmup
    alpha=0.0, # The minimum learning rate at the end of the decay
    warmup_target=initial_learning_rate,
    warmup_steps=warmup_steps
)


# --- 2. Re-compile the Model with the New Learning Rate Schedule ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)


# --- 3. Define a More Patient EarlyStopping Callback ---
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc',
    patience=15, # Even more patience to let the new LR schedule work
    mode='max',
    restore_best_weights=True
)


# --- 4. Train the Model with the New Strategy ---
print("--- Starting Advanced Training with LR Warm-up ---")
history = model.fit(
    train_ds,
    epochs=100, # Set a high limit; EarlyStopping will find the best
    validation_data=val_ds,
    class_weight=class_weight_dict,
    callbacks=[early_stopping]
)


# --- 5. Evaluate with Test-Time Augmentation (TTA) ---
print("\n--- Evaluating with Test-Time Augmentation for Higher Accuracy ---")
tta_predictions = []
tta_steps = 10

# Create a new dataset from the validation set, unbatched
unbatched_val_ds = val_ds.unbatch()

for image, label in unbatched_val_ds:
    # Get predictions for multiple augmented versions of the same image
    augmented_images = tf.image.random_flip_left_right(tf.expand_dims(image, 0))
    augmented_images = tf.image.random_flip_up_down(augmented_images)
    # Add more augmentations here if needed...
    
    # Repeat the image and augment `tta_steps` times
    predictions = model.predict(tf.repeat(augmented_images, tta_steps, axis=0), verbose=0)
    
    # Average the predictions
    tta_predictions.append(np.mean(predictions))

# Convert to a NumPy array for easier processing
tta_predictions = np.array(tta_predictions)
true_labels = np.concatenate([y for x, y in val_ds], axis=0)

# Calculate final metrics with TTA
tta_auc = tf.keras.metrics.AUC()
tta_auc.update_state(true_labels, tta_predictions)

print(f"\nFinal Validation Accuracy (Standard): {model.evaluate(val_ds, verbose=0)[1]*100:.2f}%")
print(f"Final Validation AUC (Standard): {model.evaluate(val_ds, verbose=0)[2]:.4f}")
print("-" * 30)
print(f"Final Validation AUC (with TTA): {tta_auc.result().numpy():.4f}")



--- Starting Advanced Training with LR Warm-up ---
Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 18s 561ms/step - accuracy: 0.6092 - auc: 0.6108 - loss: 0.6836 - val_accuracy: 0.6197 - val_auc: 0.7136 - val_loss: 0.6558
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 367ms/step - accuracy: 0.5493 - auc: 0.5799 - loss: 0.6877 - val_accuracy: 0.6620 - val_auc: 0.7138 - val_loss: 0.6381
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 376ms/step - accuracy: 0.5968 - auc: 0.6548 - loss: 0.6541 - val_accuracy: 0.6901 - val_auc: 0.7220 - val_loss: 0.6329
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 357ms/step - accuracy: 0.5739 - auc: 0.6111 - loss: 0.6747 - val_accuracy: 0.6831 - val_auc: 0.7359 - val_loss: 0.6247
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 360ms/step - accuracy: 0.6056 - auc: 0.6484 - loss: 0.6613 - val_accuracy: 0.6831 - val_auc: 0.7579 - val_loss: 0.6220
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 366ms/step - accuracy: 0.6004 - auc: 0.6563 - loss: 0.6553 - val_accuracy: 0.6901 - val_auc: 0.7609 -


RESNET


In [5]:
import tensorflow as tf

# --- Define the ResNet50V2 model ---
# We use the same data pipeline and image size (224x224)
IMG_SHAPE_RESNET = (224, 224, 3)
inputs_resnet = tf.keras.Input(shape=IMG_SHAPE_RESNET)

# Apply the same augmentation
x_resnet = data_augmentation(inputs_resnet)
x_resnet = tf.keras.applications.resnet_v2.preprocess_input(x_resnet)

base_model_resnet = tf.keras.applications.ResNet50V2(
    include_top=False, weights='imagenet', input_shape=IMG_SHAPE_RESNET
)
base_model_resnet.trainable = False

x_resnet = base_model_resnet(x_resnet, training=False)
x_resnet = tf.keras.layers.GlobalAveragePooling2D()(x_resnet)
x_resnet = tf.keras.layers.Dropout(0.3)(x_resnet)
outputs_resnet = tf.keras.layers.Dense(1, activation='sigmoid')(x_resnet)

model_resnet = tf.keras.Model(inputs_resnet, outputs_resnet)

# --- Compile and Train the ResNet50V2 model ---
model_resnet.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

print("--- Starting Training for the Second Base Model (ResNet50V2) ---")
history_resnet = model_resnet.fit(
    train_ds,
    epochs=50,
    validation_data=val_ds,
    class_weight=class_weight_dict,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=8, mode='max', restore_best_weights=True)]
)


94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 36s 0us/step
--- Starting Training for the Second Base Model (ResNet50V2) ---
Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 789ms/step - accuracy: 0.5158 - auc: 0.5020 - loss: 0.7930 - val_accuracy: 0.5352 - val_auc: 0.6692 - val_loss: 0.7376
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 754ms/step - accuracy: 0.5158 - auc: 0.5116 - loss: 0.7968 - val_accuracy: 0.5704 - val_auc: 0.6298 - val_loss: 0.6642
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 757ms/step - accuracy: 0.4965 - auc: 0.5046 - loss: 0.7701 - val_accuracy: 0.5704 - val_auc: 0.6199 - val_loss: 0.6538
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 749ms/step - accuracy: 0.5070 - auc: 0.5129 - loss: 0.7478 - val_accuracy: 0.5775 - val_auc: 0.6315 - val_loss: 0.6505
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 650ms/step - accuracy: 0.5352 - auc: 0.5367 - loss: 0.7529 - val_accuracy: 0.6479 - val_auc: 0.6431 - val_loss: 0.6540
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 11s 594ms/step - accuracy: 0.5440 - auc:

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np

# --- Generate Predictions from Both Base Models ---
print("\nGenerating predictions from base models...")
train_preds_effnet = model.predict(train_ds)
val_preds_effnet = model.predict(val_ds)

train_preds_resnet = model_resnet.predict(train_ds)
val_preds_resnet = model_resnet.predict(val_ds)

# --- Create the Meta-Dataset ---
X_train_meta = np.hstack([train_preds_effnet, train_preds_resnet])
X_val_meta = np.hstack([val_preds_effnet, val_preds_resnet])

y_train_meta = np.concatenate([y for x, y in train_ds])
y_val_meta = np.concatenate([y for x, y in val_ds])

# --- Train the Meta-Classifier ---
print("Training the meta-classifier...")
meta_model = LogisticRegression()
meta_model.fit(X_train_meta, y_train_meta)

# --- Evaluate the Final Ensemble Model ---
final_predictions = meta_model.predict_proba(X_val_meta)[:, 1]
ensemble_auc = roc_auc_score(y_val_meta, final_predictions)
ensemble_accuracy = accuracy_score(y_val_meta, final_predictions > 0.5)

print("\n--- Stacking Ensemble Performance ---")
print(f"Final Ensemble Validation Accuracy: {ensemble_accuracy*100:.2f}%")
print(f"Final Ensemble Validation AUC: {ensemble_auc:.4f}")



Generating predictions from base models...
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 265ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 150ms/step
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 485ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 383ms/step
Training the meta-classifier...

--- Stacking Ensemble Performance ---
Final Ensemble Validation Accuracy: 76.06%
Final Ensemble Validation AUC: 0.8315


In [8]:
import joblib

# --- Save the Keras base models in the recommended .keras format ---
model.save('efficientnet_model.keras')
model_resnet.save('resnet_model.keras')

# --- Save the Scikit-learn meta-model ---
joblib.dump(meta_model, 'meta_model.joblib')

print("All three models have been saved successfully:")
print("Files created: efficientnet_model.keras, resnet_model.keras, meta_model.joblib")



All three models have been saved successfully:
Files created: efficientnet_model.keras, resnet_model.keras, meta_model.joblib
